# TWITTER SENTIMENT ANALYSIS
## SoAI - Week 4 - Navneet Prakash Dubey

## Refrences
- [Video Guide](https://youtu.be/4YGkfAd2iXM)
- [Sentiment 140 Dataset](https://www.kaggle.com/datasets/kazanova/sentiment140)

In [ ]:
!pip install kaggle

> ⚠️ Run this after  uploading your `kaggle.json` file!

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

## Importing the Sentiment 140 Dataset from Kaggle using API

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 75% 61.0M/80.9M [00:00<00:00, 169MB/s]
100% 80.9M/80.9M [00:00<00:00, 151MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print(f'✅ The dataset was successfully extracted!')

The dataset was successfully extracted!


In [ ]:
import numpy as np
import pandas as pd
import re  #for regex
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split # For spliting the dataset into traning and testing set
from sklearn.linear_model import LogisticRegression # For training the model
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


> 🔵 Stopwords : Words that don't add any meaning to sentence.
we can remove these, because they don't add nay value while training

## Data Processing

In [ ]:
x_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
x_data.shape

(1599999, 6)

In [ ]:
x_data.head()

In [ ]:
col_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
x_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=col_names, encoding='ISO-8859-1')

In [ ]:
x_data.shape

(1600000, 6)

In [ ]:
x_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Counting the number of missing values in the dataset
x_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
# Checking the distribution of target coloumm
x_data['target'].value_counts()

,count
target,
0,800000
4,800000


## Relabling the target values
Since there are no feilds with target '2' in the dataset, we will be convert all the targets with value of '4' into 1

[ref](https://youtu.be/4YGkfAd2iXM?t=1903)

In [ ]:
x_data.replace({'target': {4:1}}, inplace=True)
print('✅ Succesfully relabled the target values!')
# x_data['target'].value_counts()

✅ Succesfully relabled the target values!


,count
target,
0,800000
1,800000


## Stemming
> It's the process of reducing a word into it's root word

In [ ]:
pStem = PorterStemmer()


In [ ]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # Removes anything that is not an alphabet
  stemmed_content = stemmed_content.lower().split()
  # we will now remove all the stop words
  stemmed_content = [pStem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

> Description:
we recive the text coloum (from dataset) into the stemming function



In [ ]:
x_data['stemmed_content'] = x_data['text'].apply(stemming) # This will take around 50-60 Minutes

In [ ]:
x_data.head()
print(f'✅ The text field was successfully stemmed!')

✅ The text field was successfully stemmed!


## Seprating & Splitting the data

In [ ]:
X = x_data['stemmed_content'].values
Y = x_data['target'].values

In [ ]:
# Splitting the data for Training & Testing
# Test data size = 20%
# The remaining data will be training data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=5)

## Feature Extraction: Converting the textual data into numerical data

[Ref](https://youtu.be/4YGkfAd2iXM?t=3799)

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test) # we don't fit the test data!

## Traning the ML Model [Logistical Regression]

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

### Evaluating the model

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print(f'🎯Accuracy on training data : {training_data_accuracy}')

🎯Accuracy on training data : 0.78068828125


In [ ]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print(f'🎯Accuracy on testing data : {testing_data_accuracy}')

🎯Accuracy on testing data : 0.772940625


## Exporting the trained model

In [ ]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb')) # wb -> Write as binary

## Loading the saved model for future predictions

In [ ]:
# loaded_model = pickle.load(open(filename, 'rb')) # rb -> Read as binary
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [ ]:
X_new = X_test[400]
prediction = loaded_model.predict(X_new)
print(prediction)
if prediction[0] == 0:
  print('This tweet is negative')
else:
  print('This tweet is positive')

[0]
This tweet is negative
